In [1]:
# hide pakcage non critical warnings

import warnings

warnings.filterwarnings('ignore')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))


%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import matplotlib.pyplot as plt

import scipy.io as sio

import torch
import torch.nn as nn

import torch.nn.functional as F
import torch.nn.init as init

from custom_data import DCCPT_data

from config import cfg, get_data_dir, get_output_dir, AverageMeter, remove_files_in_dir
from convSDAE import convSDAE
from tensorboard_logger import Logger

import os
import random
import numpy as np

import data_params as dp

In [3]:
import devkit.api as dk

In [4]:
net = convSDAE(dim=[1, 50, 50, 50, 10], output_padding=[0, 1, 0], numpen=4, dropout=0.2, slope=0)

In [5]:
net

convSDAE(
  (enc_0): Conv2d(1, 50, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (benc_0): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dec_0): ConvTranspose2d(50, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (bdec_0): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (enc_1): Conv2d(50, 50, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (benc_1): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dec_1): ConvTranspose2d(50, 50, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), output_padding=(1, 1))
  (bdec_1): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (enc_2): Conv2d(50, 50, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (benc_2): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dec_2): ConvTranspose2d(50, 50, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (bdec_2): BatchNorm

In [6]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

In [7]:
lr =0.0001

In [8]:
numlayers = 4
lr = 10
maxepoch = 2
stepsize = 10

In [9]:
for par in net.base[numlayers-1].parameters():
    par.requires_grad = True

In [10]:
for par in net.bbase[numlayers-1].parameters():
    par.requires_grad = True
for m in net.bbase[numlayers-1].modules():
    if isinstance(m, nn.BatchNorm2d):
        m.training = True

In [11]:
# setting up optimizer - the bias params should have twice the learning rate w.r.t. weights params
bias_params = filter(lambda x: ('bias' in x[0]) and (x[1].requires_grad), net.named_parameters())
bias_params = list(map(lambda x: x[1], bias_params))
nonbias_params = filter(lambda x: ('bias' not in x[0]) and (x[1].requires_grad), net.named_parameters())
nonbias_params = list(map(lambda x: x[1], nonbias_params))

In [12]:
optimizer = optim.SGD([{'params': bias_params, 'lr': 2*lr}, {'params': nonbias_params}],
                      lr=lr, momentum=0.9, weight_decay=0.0, nesterov=True)

scheduler = lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.1)

In [13]:
from torch.utils.tensorboard import SummaryWriter


In [43]:
writer = SummaryWriter('fashion_mnist_experiment_1')


In [15]:
datadir = get_data_dir("cmnist")


In [16]:
trainset = DCCPT_data(root=datadir, train=True, h5=False)

In [17]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True)


In [18]:
dataiter = iter(trainloader)

In [19]:
dataiter.next()

[tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]],
 
 
         [[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]],
 
 
         [[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]],
 
 
         ...,
 
 
         [[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ..

In [20]:
images, labels = dataiter.next()


In [21]:
images.shape

torch.Size([256, 1, 28, 28])

In [22]:
torch.tensor(3)

tensor(3)

In [28]:
labels

tensor([7, 3, 9, 3, 2, 8, 0, 4, 6, 7, 5, 3, 4, 0, 1, 3, 4, 6, 3, 3, 9, 5, 1, 4,
        0, 7, 6, 2, 3, 1, 8, 5, 4, 0, 1, 8, 0, 5, 0, 0, 6, 2, 6, 3, 8, 6, 1, 9,
        9, 2, 1, 3, 5, 5, 3, 9, 4, 7, 5, 0, 9, 9, 3, 0, 1, 3, 7, 8, 2, 6, 2, 2,
        0, 6, 8, 2, 3, 0, 9, 0, 1, 9, 2, 8, 6, 2, 0, 9, 2, 2, 6, 6, 9, 4, 3, 5,
        7, 3, 8, 1, 7, 9, 7, 1, 0, 8, 1, 2, 1, 8, 2, 6, 9, 1, 6, 7, 5, 9, 0, 5,
        2, 9, 9, 8, 8, 6, 8, 5, 5, 6, 5, 0, 6, 5, 8, 8, 9, 9, 4, 5, 2, 8, 0, 2,
        1, 6, 2, 2, 4, 3, 3, 8, 9, 1, 0, 7, 8, 0, 2, 4, 5, 5, 6, 1, 7, 0, 6, 4,
        4, 0, 7, 0, 4, 1, 9, 8, 4, 3, 6, 2, 3, 9, 4, 5, 0, 8, 1, 2, 6, 7, 3, 4,
        3, 3, 7, 7, 2, 9, 5, 7, 5, 1, 7, 5, 1, 3, 6, 0, 5, 0, 0, 4, 2, 2, 9, 2,
        9, 7, 5, 5, 2, 8, 1, 5, 1, 1, 3, 1, 7, 3, 4, 6, 2, 6, 3, 3, 9, 2, 5, 9,
        0, 2, 7, 5, 0, 7, 4, 0, 2, 6, 6, 4, 6, 4, 7, 0], dtype=torch.uint8)

In [58]:
net(images, torch.tensor(1))

tensor(0.7563, grad_fn=<MeanBackward0>)

In [59]:
writer = SummaryWriter('fashion_mnist_experiment_1')


In [61]:
writer.add_graph(net, (images, torch.tensor(4)))
# writer.close()